In [132]:
import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVR

from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials

import mlflow
import findspark

In [133]:
findspark.init()

In [134]:
Xtreino = np.load("Xtreino5.npy")
Ytreino = np.load("ytreino5.npy")

Xteste = np.load("Xteste5.npy")
Yteste = np.load("yteste5.npy")

def erro_absoluto_medio(X, Y): #Verificar se isso ta certo
  n = np.size(X)
  return (np.sum(abs(X-Y)))/n

In [135]:
def objective(params):
    clf = SVR(kernel='rbf')
    accuracy = cross_val_score(clf, Xtreino, Ytreino).mean()
    return {'loss': accuracy, 'status': STATUS_OK}

In [137]:
search_space = hp.choice('classifier_type' , [
    {
        'type': 'naive_bayes',
    },
    {
        #'type': 'svm',
        'C': hp.loguniform('C', np.log(2**-5), np.log(2**15)),
        #'C': hp.uniform('C', 2e-5, 2e15),
        #'gamma': hp.uniform('gamma', 2e-15, 2e3),
        'gamma': hp.loguniform('gamma', np.log(2**-15), np.log(2**3)),
        'epsilon': hp.uniform('epsilon', 0.05, 1.0),
        #'kernel': 'rbf'
    }
])

In [138]:
algo=tpe.suggest

spark_trials = SparkTrials(parallelism=2)#, timeout=10000)

In [139]:
with mlflow.start_run():
    best_result = fmin(fn=objective, space=search_space, algo=algo, max_evals=125, trials=spark_trials)


 99%|█████████▉| 124/125 [02:08<00:01,  1.04s/trial, best loss: 0.20514321425759513]


Total Trials: 125: 125 succeeded, 0 failed, 0 cancelled.


In [140]:
    print('best parameters:')
    print(best_result)

best parameters:
{'C': 0.1716706997904996, 'classifier_type': 1, 'epsilon': 0.8019768761573266, 'gamma': 0.12328797843241332}


In [136]:
print(clf)

NameError: name 'clf' is not defined